In [5]:
## 模型与剖分

import math
import numpy as np
from numpy.linalg import solve
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt

from tool import PDE, getIsBdNode, uniform_refine, get_cr_node_cell
from tool import get_A1_A2_F, my_solve, H1Error, print_error, print_P
from tool import get_stiff_and_div_matrix, get_bb

n   = 5  #剖分次数
n   = n + 1 
Lam = [1,1e4,1e8]
Mu  = Lam

H     = np.zeros(n)                              #步长
P     = np.zeros(len(Lam))                       #误差阶
E     = np.zeros((len(Lam),n), dtype=np.float64) #每个lambda(行)对应的误差(列)
    
for i in range(len(Lam)):
    pde = PDE(1, Lam[i])# / 2 + 10, Lam[i])
    node = np.array([
            (0,0),
            (1,0),
            (1,1),
            (0,1)], dtype=np.float64)
    cell = np.array([(1,2,0), (3,0,2)], dtype=np.int64)
    for j in range(n):
        NN = node.shape[0]
        NC = cell.shape[0]
        cm = np.ones(NC, dtype=np.float64) / NC
     
        # 单元刚度矩阵和单元载荷向量 
        A1, A2 = get_stiff_and_div_matrix(node, cell, cm)
        bb = get_bb(pde, node, cell, cm)
        
        A1, A2, F = get_A1_A2_F(A1, A2, bb, node, cell)
        A = pde.mu * A1 + (pde.lam + pde.mu) * A2
              
        uh = my_solve(A, F, node, getIsBdNode)
        u = pde.solution(node)
        H[j] = np.sqrt(2 * cm[0])
        #print("h= ", np.sqrt(2 * cm[0]))
        # 计算误差
        E[i][j] = H1Error(u, uh)
        #E[i][j] = H_1_norm(u-uh, H[j])
        #print("u= ", u)
        #print("uh= ", uh)
        node, cell = uniform_refine(node, cell)

if n-1 > 1: 
    for i in range(len(Lam)):
        fig = plt.figure()
        plt.plot(np.log(H[1:]), np.log(E[i][1:]))
        plt.title("lam={}".format(Lam[i]))
        plt.xlabel("log(h)")
        plt.ylabel("log(e)")
        plt.savefig(fname="Fem/elasticityFemLam_{}.png"
                    .format(Lam[i]))
        plt.close(fig)
        
    for i in range(len(Lam)):
        f = np.polyfit(np.log(H[1:]), np.log(E[i][1:]) ,1)
        P[i] = f[0]

#print("u= ", np.max(u))
#print("uh= ", np.max(uh))
print_error(Lam, H, E)
#print_error(Lam, H, E[isInterNode], E_rel[isInterNode])
if n-1 > 1:
    print_P(Lam, P)

---------------------Lam= 1---------------------

h=  1.0
e= 0.0

h=  0.5
e= 0.11935462553808

h=  0.25
e= 0.06600434279613125

h=  0.125
e= 0.033524494459334836

h=  0.0625
e= 0.016823662478270256

h=  0.03125
e= 0.00842002354123198


---------------------Lam= 10000.0---------------------

h=  1.0
e= 0.0

h=  0.5
e= 0.24479742009342947

h=  0.25
e= 0.11649958091039243

h=  0.125
e= 0.056559074845972626

h=  0.0625
e= 0.02715759774880131

h=  0.03125
e= 0.013556852249637758


---------------------Lam= 100000000.0---------------------

h=  1.0
e= 0.0

h=  0.5
e= 0.2448722174670959

h=  0.25
e= 0.11654704589336998

h=  0.125
e= 0.05659682917901433

h=  0.0625
e= 0.027171962631282146

h=  0.03125
e= 0.013736927382462124


---------------------误差阶---------------------
lam=  1
p=  0.9622641941815836

lam=  10000.0
p=  1.0449885407447947

lam=  100000000.0
p=  1.0412517455740105



In [9]:
## 模型与剖分

import math
import numpy as np
from numpy.linalg import solve
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt

from tool import PDE2, getIsBdNode, uniform_refine, get_cr_node_cell
from tool import get_A1_A2_F, my_solve, H1Error, print_error, print_P
from tool import get_stiff_and_div_matrix, get_bb

n   = 5  #剖分次数
n   = n + 1 
Lam = [1,1e4,1e8]
Mu  = Lam

H     = np.zeros(n)                              #步长
P     = np.zeros(len(Lam))                       #误差阶
E     = np.zeros((len(Lam),n), dtype=np.float64) #每个lambda(行)对应的误差(列)
    
for i in range(len(Lam)):
    pde = PDE2(Mu[i] / 2 + 1000, Lam[i])# / 2 + 10, Lam[i])
    node = np.array([
            (0,0),
            (1,0),
            (1,1),
            (0,1)], dtype=np.float64)
    cell = np.array([(1,2,0), (3,0,2)], dtype=np.int64)
    for j in range(n):
        NN = node.shape[0]
        NC = cell.sha/ 2 + 2000
        A1, A2, F = get_A1_A2_F(A1, A2, bb, node, cell)
        A = pde.mu * A1 + (pde.lam + pde.mu) * A2
              
        uh = my_solve(A, F, node, getIsBdNode)
        u = pde.solution(node)
        H[j] = np.sqrt(2 * cm[0])
        #print("h= ", np.sqrt(2 * cm[0]))
        # 计算误差
        E[i][j] = H1Error(u, uh)
        #E[i][j] = H_1_norm(u-uh, H[j])
        #print("u= ", u)
        #print("uh= ", uh)
        node, cell = uniform_refine(node, cell)

if n-1 > 1: 
    for i in range(len(Lam)):
        fig = plt.figure()
        plt.plot(np.log(H[1:]), np.log(E[i][1:]))
        plt.title("lam={}".format(Lam[i]))
        plt.xlabel("log(h)")
        plt.ylabel("log(e)")
        plt.savefig(fname="Fem/elasticityFemLam_{}.png"
                    .format(Lam[i]))
        plt.close(fig)
        
    for i in range(len(Lam)):
        f = np.polyfit(np.log(H[1:]), np.log(E[i][1:]) ,1)
        P[i] = f[0]

#print("u= ", np.max(u))
#print("uh= ", np.max(uh))
print_error(Lam, H, E)
#print_error(Lam, H, E[isInterNode], E_rel[isInterNode])
if n-1 > 1:
    print_P(Lam, P)

---------------------Lam= 1---------------------

h=  1.0
e= 0.0

h=  0.5
e= 0.038023215342454245

h=  0.25
e= 0.023369453302760707

h=  0.125
e= 0.012391053290788405

h=  0.0625
e= 0.00625572881506932

h=  0.03125
e= 0.0031344109962399293


---------------------Lam= 10000.0---------------------

h=  1.0
e= 0.0

h=  0.5
e= 0.06201305255994399

h=  0.25
e= 0.03257296483550001

h=  0.125
e= 0.016392410269443193

h=  0.0625
e= 0.00817508871329465

h=  0.03125
e= 0.004085316411845559


---------------------Lam= 100000000.0---------------------

h=  1.0
e= 0.0

h=  0.5
e= 0.06543202576913905

h=  0.25
e= 0.033934233918307

h=  0.125
e= 0.016978837422529508

h=  0.0625
e= 0.008456637261956458

h=  0.03125
e= 0.004225025416835104


---------------------误差阶---------------------
lam=  1
p=  0.9102602638211592

lam=  10000.0
p=  0.9842473112630415

lam=  100000000.0
p=  0.991051546491601



In [7]:
## 模型与剖分

import math
import numpy as np
from numpy.linalg import solve
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt

from tool import PDE3, getIsBdNode, uniform_refine, get_cr_node_cell
from tool import get_A1_A2_F, my_solve, H1Error, print_error, print_P
from tool import get_stiff_and_div_matrix, get_bb

n   = 5  #剖分次数
n   = n + 1 
Lam = [1,1e4,1e8]
Mu  = Lam

H     = np.zeros(n)                              #步长
P     = np.zeros(len(Lam))                       #误差阶
E     = np.zeros((len(Lam),n), dtype=np.float64) #每个lambda(行)对应的误差(列)
    
for i in range(len(Lam)):
    pde = PDE3(1, Lam[i])# / 2 + 10, Lam[i])
    node = np.array([
            (0,0),
            (1,0),
            (1,1),
            (0,1)], dtype=np.float64)
    cell = np.array([(1,2,0), (3,0,2)], dtype=np.int64)
    for j in range(n):
        NN = node.shape[0]
        NC = cell.shape[0]
        cm = np.ones(NC, dtype=np.float64) / NC
     
        # 单元刚度矩阵和单元载荷向量 
        A1, A2 = get_stiff_and_div_matrix(node, cell, cm)
        bb = get_bb(pde, node, cell, cm)
        
        A1, A2, F = get_A1_A2_F(A1, A2, bb, node, cell)
        A = pde.mu * A1 + (pde.lam + pde.mu) * A2
              
        uh = my_solve(A, F, node, getIsBdNode)
        u = pde.solution(node)
        H[j] = np.sqrt(2 * cm[0])
        #print("h= ", np.sqrt(2 * cm[0]))
        # 计算误差
        E[i][j] = H1Error(u, uh)
        #E[i][j] = H_1_norm(u-uh, H[j])
        #print("u= ", u)
        #print("uh= ", uh)
        node, cell = uniform_refine(node, cell)

if n-1 > 1: 
    for i in range(len(Lam)):
        fig = plt.figure()
        plt.plot(np.log(H[1:]), np.log(E[i][1:]))
        plt.title("lam={}".format(Lam[i]))
        plt.xlabel("log(h)")
        plt.ylabel("log(e)")
        plt.savefig(fname="Fem/elasticityFemLam_{}.png"
                    .format(Lam[i]))
        plt.close(fig)
        
    for i in range(len(Lam)):
        f = np.polyfit(np.log(H[1:]), np.log(E[i][1:]) ,1)
        P[i] = f[0]

#print("u= ", np.max(u))
#print("uh= ", np.max(uh))
print_error(Lam, H, E)
#print_error(Lam, H, E[isInterNode], E_rel[isInterNode])
if n-1 > 1:
    print_P(Lam, P)

---------------------Lam= 1---------------------

h=  1.0
e= 0.0

h=  0.5
e= 0.24719273157146132

h=  0.25
e= 0.1210255221250261

h=  0.125
e= 0.0594389584850562

h=  0.0625
e= 0.029577901521539177

h=  0.03125
e= 0.014774684757096763


---------------------Lam= 10000.0---------------------

h=  1.0
e= 0.0

h=  0.5
e= 0.5299086061335296

h=  0.25
e= 0.24911297589016967

h=  0.125
e= 0.11771484218028216

h=  0.0625
e= 0.05644378533350253

h=  0.03125
e= 0.02666842840176844


---------------------Lam= 100000000.0---------------------

h=  1.0
e= 0.0

h=  0.5
e= 0.5300770328830062

h=  0.25
e= 0.24925365876838296

h=  0.125
e= 0.11784798354669486

h=  0.0625
e= 0.05656752642212566

h=  0.03125
e= 0.026719861011132728


---------------------误差阶---------------------
lam=  1
p=  1.0161593851712856

lam=  10000.0
p=  1.0766991212483263

lam=  100000000.0
p=  1.0760003939608977

